In [1]:
import subprocess
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.interval import IntervalTrigger
import atexit

from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin
import pandas as pd
import numpy as np

import covid19_model as model

In [2]:
access_count = 0

def run_training():
    model.fetch_data()
    model.detect_growth('data/covid19_data.csv', 'data/covid19_processed_data_', False)
    model.detect_growth('data/covid19_data_backtesting.csv', 'data/covid19_processed_backtesting_data_', True)
    model.calculate_forecast()
    
scheduler = BackgroundScheduler()
scheduler.start()

scheduler.add_job(
    func=run_training,
    trigger='cron',
    hour='2', 
    minute='00')

atexit.register(lambda: scheduler.shutdown())

Growth increasing: Afghanistan_cases | Fastest grow day: 82.2722969265232 | Infections: 627.1625996744144
Growth stabilized: Albania_cases | Fastest grow day: 70.14263098143321 | Infections: 254.46648425388358
Growth stabilized: Algeria_cases | Fastest grow day: 78.08099652344993 | Infections: 1598.1749105787387
Growth stabilized: Andorra_cases | Fastest grow day: 70.136474536702 | Infections: 363.0137791070213
Growth increasing: Angola_cases | Fastest grow day: 83.35930772930286 | Infections: 26.706282571887154
Growth stabilized: Antigua and Barbuda_cases | Fastest grow day: 77.30392839195282 | Infections: 17.296187051931124
Growth stabilized: Argentina_cases | Fastest grow day: 74.57523059214626 | Infections: 1396.481184308679
Growth stabilized: Armenia_cases | Fastest grow day: 71.515637792887 | Infections: 609.4541261869778
Growth stabilized: Australia_australian capital territory_cases | Fastest grow day: 67.56181358414823 | Infections: 60.6528878008917
Growth stabilized: Australi

/opt/conda/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.



Processed: Afghanistan_cases
Processed: Albania_cases
Processed: Algeria_cases
Processed: Andorra_cases
Processed: Angola_cases
Processed: Antigua and Barbuda_cases
Processed: Argentina_cases
Processed: Armenia_cases
Processed: Australia_australian capital territory_cases
Processed: Australia_new south wales_cases
Processed: Australia_northern territory_cases
Processed: Australia_queensland_cases
Processed: Australia_south australia_cases
Processed: Australia_tasmania_cases
Processed: Australia_victoria_cases
Processed: Australia_western australia_cases
Processed: Austria_cases
Processed: Azerbaijan_cases
Processed: Bahamas_cases
Processed: Bahrain_cases
Processed: Bangladesh_cases
Processed: Barbados_cases
Processed: Belarus_cases
Processed: Belgium_cases
Processed: Benin_cases
Processed: Bhutan_cases
Processed: Bolivia_cases
Processed: Bosnia_cases
Processed: Brazil_cases
Processed: Brunei_cases
Processed: Bulgaria_cases
Processed: Burkina Faso_cases
Processed: Cabo Verde_cases
Proce

INFO:apscheduler.scheduler:Scheduler started
INFO:apscheduler.scheduler:Added job "run_training" to job store "default"


Processed: Yemen_cases
Forecast calculation completed


<function __main__.<lambda>()>

In [3]:
app = Flask(__name__)
CORS(app)

@app.route("/katana-ml/api/v1.0/forecast/covid19", methods=['POST'])
def forecast():
    global access_count
    access_count += 1
    
    country = request.json['country']
    
    df = pd.read_csv('data/covid19_forecast_data_' + country + '.csv', parse_dates=True)
    df = df.drop(df.columns[[0]], axis=1)
    
    result = df.to_json(orient='records', date_format='iso')
    return result

@app.route("/katana-ml/api/v1.0/forecast/covid19/stats", methods=['GET'])
def stats():
    df = pd.read_csv('data/covid19_stats_countries.csv', parse_dates=True)
    df = df.drop(df.columns[[0]], axis=1)
    
    result = df.to_json(orient='records', date_format='iso')
    return result

@app.route("/katana-ml/api/v1.0/forecast/covid19/count", methods=['GET'])
def accecc_count():
    global access_count
    return str(access_count)

@app.route("/katana-ml/api/v1.0/forecast/covid19/countries", methods=['GET'])
def get_countries():
    df = pd.read_csv('data/covid19_countries_list.csv')
    df = df.drop(df.columns[[0]], axis=1)
    
    result = df.to_json(orient='records', date_format='iso')
    return result

# running REST interface port=5001
if __name__ == "__main__":
    app.run(debug=False, host='0.0.0.0', port=5001)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://0.0.0.0:3000/ (Press CTRL+C to quit)
INFO:werkzeug:172.17.0.1 - - [11/Apr/2020 10:47:42] "OPTIONS /katana-ml/api/v1.0/forecast/covid19/stats HTTP/1.1" 200 -
INFO:werkzeug:172.17.0.1 - - [11/Apr/2020 10:47:42] "OPTIONS /katana-ml/api/v1.0/forecast/covid19/countries HTTP/1.1" 200 -
INFO:werkzeug:172.17.0.1 - - [11/Apr/2020 10:47:42] "OPTIONS /katana-ml/api/v1.0/forecast/covid19 HTTP/1.1" 200 -
INFO:werkzeug:172.17.0.1 - - [11/Apr/2020 10:47:42] "GET /katana-ml/api/v1.0/forecast/covid19/stats HTTP/1.1" 200 -
INFO:werkzeug:172.17.0.1 - - [11/Apr/2020 10:47:42] "GET /katana-ml/api/v1.0/forecast/covid19/countries HTTP/1.1" 200 -
INFO:werkzeug:172.17.0.1 - - [11/Apr/2020 10:47:42] "POST /katana-ml/api/v1.0/forecast/covid19 HTTP/1.1" 200 -
INFO:werkzeug:172.17.0.1 - - [11/Apr/2020 10:47:43] "POST /katana-ml/api/v1.0/forecast/covid19 HTTP/1.1" 200 -
INFO:werkzeug:172.17.0.1 - - [11/Apr/2020 10:47:58] "OPTIONS /katana-ml/api/v1.0/forecast/covid19/stats HTTP/1.1